# Computations in Dist(G) for G Unipotent

#    Michael Crumley

#    mikecrumley@hotmail.com

##   for python version 3.5

This is a companion file to the paper "Postive Characteristic Representations and Distribution Algebras of Certain Unipotent Algebraic Groups", which can be found at

https://github.com/mikecrumley/Computations-in-Dist-G-for-G-Unipotent/blob/master/Positive%20Characteristic%20Reps_Michael%20Crumley.pdf

It provides code for doing computations in $\text{Dist}(U_n)$ ($n$ arbitrary), the distribution algebra of $U_n$, where $U_n$ is the space of all $n \times n$ unipotent upper triangular matrices over a field $k$:  

\begin{equation*}
\left(
\begin{array}{ccccc}
  1 & x_{12} & x_{13} & \ldots & x_{1n} \\
   & 1 & x_{23} & \ldots & x_{2n} \\
   &  & \ddots & \ddots & \vdots \\
   &  &  & 1 & x_{n-1,n} \\
   &  &  &  & 1 \\
\end{array}
\right)
\end{equation*}

A complete understanding of the the paper is by no means necessary.  What is necessary is an understanding of how elements of $\text{Dist}(U_n)$ are defined (proposition 3.1), that is, as finite $k$-linear combinations of the basis elements $\alpha(M)$, where $M$ is a strictly upper triangular $n \times n$ matrix with non-negative integer entries.

The multiplication rule for two basis elements $\alpha(M)$ and $\alpha(N)$ can be found in proposition 3.3.  However, the implementation we give instead relies on lemma 6.6, along with the row-wise decomposition given in proposition 3.5, which we think is probably near optimal in terms of speed (in arbitrary characteristic that is; in fixed positive characteristic, there are ways to speed this up significantly, for instance using proposition 3.12 (3),  but this is not done here and is not strictly necessary, since we have the ```mod_p``` function described below).  However, all of this is placed in the background, and so understanding multiplication in $\text{Dist}(U_n)$ is not actually necessary (but would obviously be helpful for interpreting the results of your computations).

Since any unipotent algebraic group $G$ can be embedded in $U_n$ for some $n$, in fact $\text{Dist}(G) \subset \text{Dist}(U_n)$ for some $n$ (see section 2), and so in principle this implementation can be used for doing computations in any $\text{Dist}(G)$ for $G$ unipotent.  Once an embedding $G \subset U_n$ is realized, the embedding $\text{Dist}(G) \subset \text{Dist}(U_n)$ can be realized rather straightforwardly; see section 9.1 for an example of how the defining polynomials of $G$ (polynomial relations among the entries of $U_n$) correspond to \textit{linear} relations among the basis elements of $\text{Dist}(U_n)$.

If you find this useful, please shoot me a line at mikecrumley@hotmail.com.

To begin, load the following code (or copy and paste it to your favorite Python interpreter), which the author can only guarantee will work in python 3.5:

In [1]:
# implementation of "dist" class
# successfully tested in python 3.5; not guaranteed to work in other python versions

import \
    numbers, \
    collections.abc

def my_list_prod(L):
    ans = 1
    for x in L:
        ans = ans*x
    return ans

def my_factorial(n):
    ans = 1
    L = [i for i in range(1,n+1)]
    return my_list_prod(L)

def multinomial(*args):  #non-negative integers (sum not included in args)
    if not all([type(k) is int for k in args]):
        raise RuntimeError("Error, in multinomial, non-integer argument")
    fact_list = [my_factorial(k) for k in args]
    n = sum(args)
    return(my_factorial(n) // (my_list_prod(fact_list)))

def remove_duplicates(L):     #L is a list.  order won't change.  Need a different method for non-hashable types
    are_hashable = all([isinstance(x,collections.Hashable) for x in L])
    if are_hashable:
        seen = set()
        seen_add = seen.add
        return [x for x in L if not (x in seen or seen_add(x))]
    else:
        output = []
        for x in L:
            if x not in output:
                output.append(x)
        return output

def my_cart_prod(L):   #L is a list of non-negative integers.  It's a generator (use 'next')
    Q = [0 for i in L]
    yield Q
    def next_one(ans, L):
        Q = ans
        n = len(L)
        if Q[0] < L[0]:
            Q[0] = Q[0] + 1
            return (Q)
        else:
            return [0] + next_one(Q[1:n], L[1:n])
    while Q != L:
        Q = next_one(Q,L)
        yield Q
    yield "done"     #use to test if done (instead of rasing an exception)

def all_indices(L,elem):    #indices for elem in list L
    return [i for i, x in enumerate(L) if x == elem]

def get_items(L,indices):   #items from L having certain indices (a list)
    return list(map(L.__getitem__, indices))

def are_equal_as_sets(L1,L2):    #tests if two lists are equal, after converting to sets (unordered, duplicates removed).  useful for non-hashable types, like dictionaries (which can't be converted to sets)
    L1_removed = remove_duplicates(L1)
    L2_removed = remove_duplicates(L2)
    if len(L1_removed) != len(L2_removed):
        return False
    ans = True
    for i in range(0,len(L1_removed)):
        if L1_removed[i] in L2_removed:
            continue
        else:
            ans = False
            break
    return ans

def remove_zeroes_from_dict(aDict):
    keys = list(aDict)
    new_dict = {}
    for key in keys:
        if aDict[key] != 0:
            new_dict[key] = aDict[key]
    return new_dict

def dict_sum(dict1,dict2):    #sum of two dictionaries (sum of entries)
    keys_1 = list(dict1)
    keys_2 = list(dict2)
    new_keys = remove_duplicates(keys_1 + keys_2)
    new_dict = {}
    for key in new_keys:
        new_dict[key] = get_entry(dict1,key[0],key[1]) + get_entry(dict2,key[0],key[1])
    return new_dict

def dict_diff(dict1,dict2):    #difference of two dictionaries (difference of entries)
    keys_1 = list(dict1)
    keys_2 = list(dict2)
    new_keys = remove_duplicates(keys_1 + keys_2)
    new_dict = {}
    for key in new_keys:
        new_dict[key] = get_entry(dict1,key[0],key[1]) - get_entry(dict2,key[0],key[1])
    return new_dict

def get_entry(aDict,i,j):    #get entry from a dictionary (zero returned if key not present)
    keys = list(aDict)
    if (i,j) in keys:
        return aDict[(i,j)]
    else:
        return 0

def are_valid_dicts(dicts):                #dicts is a list of dictionaires
    if not all([type(x) == dict for x in dicts]):
        return False
    dict_indices_list = [list(x) for x in dicts]
    for x in dict_indices_list:
        for y in x:
            if not (len(y) == 2 and isinstance(y[0], int) and isinstance(y[1], int) and y[0] > 0 and y[1] > y[0]):
                return False
    dict_values_list = [list(x.values()) for x in dicts]
    for x in dict_values_list:
        for y in x:
            if not (int(y) == y and y >= 0):
                return False
    return True

def max_dim(dicts):     #dicts is a list of dictionaries
    key_list = [list(x) for x in dicts]
    n = 1
    for x in key_list:
        for y in x:
            n = max(n,y[1])
    return n

def dict_multinomial(dicts):   #dicts is a list of dictionaries
    n = max_dim(dicts)
    ans = 1
    for i in range(1,n):
        for j in range(i+1,n+1):
            curr_entries = []
            for x in dicts:
                curr_entries.append(get_entry(x,i,j))
            ans = ans*multinomial(*curr_entries)
    return ans

def cart_prod_less_than(L,a):    #generate elements of cartesian product summing to no greater than a
    Q = [0]*len(L)
    n = len(L)
    curr_sum = 0
    last_index = -1
    for i in range(n-1,-1,-1):
        curr_sum = curr_sum + L[i]
        if curr_sum >= a:
            last_index = i
            final_sum = curr_sum - L[i]
            break
    if last_index == -1:
        last_tuple = L
    else:
        temp_tuple = L[(last_index + 1):n]
        temp_tuple = [a - sum(temp_tuple)]+temp_tuple
        temp_tuple = [0]*(n-len(temp_tuple)) + temp_tuple
        last_tuple = temp_tuple
    yield Q
    def next_one(ans, L):
        Q = ans
        n = len(L)
        if Q[0] < L[0] and sum(Q) < a:
            Q[0] = Q[0] + 1
            return (Q)
        else:
            return [0] + next_one(Q[1:n], L[1:n])
    while Q != last_tuple:
        Q = next_one(Q, L)
        yield Q
    yield "done"

class monomialDist:
    def __init__(self,dist_dict):   #the empty dictionary corresponds to alpha(0)
        self.dist_dict = dist_dict
        self.dimension = max_dim([dist_dict])
        if dist_dict == {}:
            self.index_list = []
        else:
            self.index_list = list(dist_dict.keys())

def alpha_generator(monom, r, s, a):
    dict = monom.dist_dict
    n = max([s,monom.dimension])
    m_list = []
    for i in range(s+1,n+1,1):
        m_list.append(get_entry(dict,s,i))
    partial_gen = cart_prod_less_than(m_list,a)
    ans = next(partial_gen)
    while ans != "done":
        yield [a - sum(ans)] + ans
        ans = next(partial_gen)
    yield "done"

def monom_mult_alpha(monom,r,s,a):      #using lemma 6.6
    dict = monom.dist_dict
    if dict == {}:             #in case monom is alpha(0)
        new_dict = {(r,s): a}
        new_dist = dist([1],[new_dict])
        return new_dist
    dim = max([monom.dimension,s])
    u_gen = alpha_generator(monom,r,s,a)
    def A_dict(U):
        A = {}
        i = 0
        for j in U:
            A[(r,s+i)] = j
            i = i+1
        return A
    def B_dict(U):
        B = dict
        temp_dict = {}
        for i in range(s+1,dim+1):
            temp_dict[(s,i)] = U[i-s]
        return dict_diff(B,temp_dict)
    ans_dist = dist([],[])
    U = next(u_gen)
    while U != "done":
        B = B_dict(U)
        A = A_dict(U)
        new_coeff = dict_multinomial([A,B])
        new_dict = dict_sum(A,B)
        new_dist = dist([new_coeff],[new_dict])
        ans_dist = ans_dist + new_dist
        U = next(u_gen)
    return ans_dist

class dist:
    def __init__(self, scalars, dicts):  #a list of scalars, an equal sized list of dictionaries.  The empty dictionary equals 1 (scalars), the empty LIST of dictionaries equals 0.
        if len(scalars) != len(dicts):
            raise RuntimeError("Error, in dist, lengths of dicts and scalars don't match")
        if not all([isinstance(x,numbers.Number) for x in scalars]):
            raise RuntimeError("Error, in dist, non-numeric type in scalars")
        if not all([type(x) == dict for x in dicts]):
            raise RuntimeError("Error, in dist, non-dict type in dicts")
        dict_indices_list = [list(x) for x in dicts]
        for x in dict_indices_list:
            for y in x:
                if not len(y) == 2 and isinstance(y[0],int) and isinstance(y[1],int) and y[0] > 0 and y[1] > y[0]:
                    raise RuntimeError("Error, in dist, invalid dictionary keys")
        dict_values_list = [list(x.values()) for x in dicts]
        for x in dict_values_list:
           for y in x:
              if not int(y) == y and y >= 0:
                 raise RuntimeError("Error, in dist, invalid dictionary values")
        dicts_zeroes_removed = [remove_zeroes_from_dict(x) for x in dicts]
        unique_dicts = remove_duplicates(dicts_zeroes_removed)
        if len(dicts_zeroes_removed) == len(unique_dicts):
            new_dicts = dicts_zeroes_removed
            new_scalars = scalars
        else:
            new_scalars = [0]*len(unique_dicts)
            i = 0
            for x in unique_dicts:
                x_indices = all_indices(dicts_zeroes_removed,x)
                x_scalars = get_items(scalars,x_indices)
                new_scalars[i] = sum(x_scalars)
                i = i+1
        i = 0
        new_new_scalars = []
        new_new_dicts = []
        for x in new_scalars:
            if x == 0:
                pass
            else:
                new_new_scalars.append(x)
                new_new_dicts.append(unique_dicts[i])
            i = i + 1
        self.scalars = new_new_scalars
        self.dicts = new_new_dicts
        self.monomials = [monomialDist(x) for x in new_new_dicts]           #this is basically private
        if len(self.monomials) == 0:
            self.dimension = 1
        else:
            self.dimension = max([x.dimension for x in self.monomials])

    def __add__(self,other):    #used when left operand is of dist type (right can be dist or numeric)
        if other == 0:
            return self
        if isinstance(other,dist):
            dist_dicts_1 = self.dicts
            dist_dicts_2 = other.dicts
            scalars_1 = self.scalars
            scalars_2 = other.scalars
            return dist(scalars_1 + scalars_2, dist_dicts_1 + dist_dicts_2)
        if not isinstance(other,numbers.Number):
            raise RuntimeError("Error, in __add__ (dist), non-dist or numeric argument")
        scalar_dist = dist([other],[{}])
        return self + scalar_dist

    def __radd__(self,other):    #used when right operand is of dist type, but left is not
        return self + other

    def dist_mult_alpha(self, r, s, a):  # relying on monom_mult_alpha
        monomials = self.monomials
        if len(monomials) == 0:
            new_dict = {(r, s): a}
            new_dist = dist([1], [new_dict])
            return new_dist
        scalars = self.scalars
        ans_dist = dist([], [])
        for i in range(0, len(monomials)):
            new_dist = monom_mult_alpha(monomials[i], r, s, a)
            new_scalars = new_dist.scalars
            new_scalars = [x * scalars[i] for x in new_scalars]
            new_dist = dist(new_scalars, new_dist.dicts)
            ans_dist = ans_dist + new_dist
        return ans_dist

    def monom_dist_mult(self, monom1):
        ans_dist = self
        monom_dict = monom1.dist_dict
        n = monom1.dimension
        for i in range(0, n):
            for j in range(n - 1, i, -1):
                r = i + 1
                s = j + 1
                a = get_entry(monom_dict,i+1,j+1)
                if a == 0:
                    pass
                else:
                    ans_dist = ans_dist.dist_mult_alpha(r, s, a)
        return ans_dist

    def dist_prod(self, dist2):
        monomials1 = self.monomials
        scalars1 = self.scalars
        ans_dist = dist([], [])
        for i in range(0, len(monomials1)):
            monom = monomials1[i]
            scalar = scalars1[i]
            new_dist = dist2.monom_dist_mult(monom)
            new_scalars = new_dist.scalars
            new_scalars = [x * scalar for x in new_scalars]
            new_dist = dist(new_scalars, new_dist.dicts)
            ans_dist = ans_dist + new_dist
        return ans_dist

    def __mul__(self,other):              #used when left operand is of dist type (right can be dist or numeric)
        if isinstance(other,dist):
            return self.dist_prod(other)
        if other == 0:
            return dist([],[])
        else:
            if not isinstance(other,numbers.Number):
                raise RuntimeError("Error, in __mul__ (dist), non-dist or numeric argument")
            scalar_dist = dist([other],[{}])
            return self.dist_prod(scalar_dist)

    def __rmul__(self,other):                        #used when right operand is of dist type, but left is not
        if not isinstance(other,numbers.Number):
            raise RuntimeError("Error, in __rmul__ (dist), non-dist or numeric argument")
        if other == 0:
            return dist([],[])
        else:
            scalar_dist = dist([other],[{}])
            return self.dist_prod(scalar_dist)

    def __pow__(self,n):
        if int(n) != n or n < 0:
            raise RuntimeError("Error, in Dist.__pow__, invalid power")
        if n == 0:
            return dist([],[])
        else:
            return self*(self**(n-1))

    def __neg__(self):
        neg_dist = dist([-1],[{}])
        return neg_dist*self

    def __sub__(self,other):
        return self + (-other)

    def __rsub__(self,other):
        return -self + other

    def __eq__(self,other):
        if isinstance(other,dist):
            self_dicts = self.dicts
            other_dicts = other.dicts
            if len(self_dicts) != len(other_dicts):
                return False
            self_scalars = self.scalars
            other_scalars = other.scalars
            self_combined_list = []
            other_combined_list = []
            for i in range(0,len(self_dicts)):
                self_combined_list.append([self_scalars[i],self_dicts[i]])
            for i in range(0,len(other_dicts)):
                other_combined_list.append([other_scalars[i],other_dicts[i]])
            return are_equal_as_sets(self_combined_list,other_combined_list)
        elif not isinstance(other,numbers.Number):
            raise RuntimeError("Error, in __eq__ (dist), non-dist or numeric argument")
        else:
            scalar_dist = dist([other],[{}])
            return self == scalar_dist
        
    def __neq__(self,other):
        return not(self == other)

    def __abs__(self):      #the (max) order of a distribution
        self_dicts = self.dicts
        max_order = 0
        for x in self.dicts:
            monom_order = sum(x.values())
            if monom_order > max_order:
                max_order = monom_order
        return max_order

    def mod_p(self,p):
        new_scalars = [x % p for x in self.scalars]
        return dist(new_scalars,self.dicts)

def alpha(dist_dict):                      #0 means alpha(0).
    if dist_dict == 0:
        return dist([1],[{}])
    if not are_valid_dicts([dist_dict]):
        raise RuntimeError("Error, in alpha, invalid dictionary")
    if isinstance(dist_dict,dict):
        return dist([1],[dist_dict])
    else:
        raise RuntimeError("Error, in alpha, invalid argument")


This defines the ``dist`` class, along all the relevant definitions for addition, multiplication, etc. for doing computations in $\text{Dist}(U_n)$.

Consider for example the strictly upper triangular matrix

\begin{equation*}
M = \left(
\begin{array}{cccc}
0 & 1 & 2 & 0 \\
 & 0 & 3 & 0 \\
 & & 0 & 1 \\
 & & & 0
\end{array}
\right)
\end{equation*}

We would like to create the distribution element $\alpha(M)$ (as an object of class ```dist```) and start doing computations with it.  This can be done using the convenience function ```alpha```:

In [2]:
dist1 = alpha({(1,2):1, (1,3):2, (2,3):3, (3,4): 1})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working


The interpretation should be clear.  The function ```alpha``` accepts as its sole argument a Python dictionary such as

In [3]:
{(1,2):1, (1,3):2, (2,3):3, (3,4):1}

{(1, 2): 1, (1, 3): 2, (2, 3): 3, (3, 4): 1}

which encodes the entries of the matrix $M$.  Note two things: first, any zero entries are not necessary to include (and the ```dist``` constructor will discard them anyway).  Second, because of this, it is not necessary to specify which $n$ in $\text{Dist}(U_n)$ you are working in.  So, if your matrix instead was

\begin{equation*}
M = \left(
\begin{array}{ccccc}
0 & 1 & 2 & 0 & 0 \\
 & 0 & 3 & 0 & 0 \\
 & & 0 & 1 & 0 \\
 & & & 0 & 0 \\
 & & & & 0 \\
\end{array}
\right)
\end{equation*}

you would declare $\alpha(M)$ the exact same way.  These routines implicitly perform computations inside the smallest $\text{Dist}(U_n)$ to which all of the relevant distributions happen to be a member.  So, really, we are doing computations not in $\text{Dist}(U_n)$ for a fixed $n$, but rather the directed union of all $\text{Dist}(U_n)$ (under the obvious inclusions).

Addition (```+```) subtraction and negation (```-```), multiplication (```*```), and exponentiation (```**```) all do what we want them to do, performing the same-named computations in $\text{Dist}(U_n)$.   For example, we can define a ```dist``` object by


In [4]:
dict1 = {(1,2):1, (1,3):2, (2,3):3, (3,4):1}
dict2 = {(1,2):2, (2,3):1}
dict3 = {(1,3):2,(2,4):1}

new_dist = (2*alpha(dict1) - 3*alpha(dict2)**2)*alpha(dict3)

and can perform computations with this new distribution ```new_dist``` just the same:

In [5]:
another_dist = 3*new_dist**3 + alpha(dict1)

The output of any valid arithmetic expression involving ```+```,```-```, ```*```, and ```**``` and involving objects of class ```dist``` is again an object of class ```dist```.  Each instance of ```dist``` has two defining attributes: a list (possibly empty) of non-zero scalars (any numerics), and an equal sized list of dictionaries, which are accessed by

In [6]:
new_dist.scalars

[12, 12, -36, -36, -27, -27]

In [7]:
new_dist.dicts

[{(3, 4): 1, (2, 3): 3, (2, 4): 1, (1, 2): 1, (1, 3): 4},
 {(3, 4): 1, (2, 3): 3, (1, 3): 4, (1, 4): 1},
 {(2, 3): 2, (2, 4): 1, (1, 2): 4, (1, 3): 2},
 {(2, 3): 2, (1, 2): 3, (1, 3): 2, (1, 4): 1},
 {(2, 3): 1, (2, 4): 1, (1, 2): 3, (1, 3): 3},
 {(2, 3): 1, (1, 2): 2, (1, 3): 3, (1, 4): 1}]

We should interpret this to mean that ```new_dist``` is equivalent to

In [8]:
new_new_dist = 12*alpha({(3, 4): 1, (2, 3): 3, (2, 4): 1, (1, 2): 1, (1, 3): 4}) \
             + 12*alpha({(3, 4): 1, (2, 3): 3, (1, 3): 4, (1, 4): 1}) \
             - 36*alpha({(2, 3): 2, (2, 4): 1, (1, 2): 4, (1, 3): 2}) \
             - 36*alpha({(2, 3): 2, (1, 2): 3, (1, 3): 2, (1, 4): 1}) \
             - 27*alpha({(2, 3): 1, (2, 4): 1, (1, 2): 3, (1, 3): 3}) \
             - 27*alpha({(2, 3): 1, (1, 2): 2, (1, 3): 3, (1, 4): 1})

which it is:

In [9]:
new_dist == new_new_dist

True

The function ```alpha``` is merely a convenience and need not be used.  An arbitrary object of class ```dist``` may be constructed by explicitly giving the scalars and dictionaries: 

In [10]:
dist2 = dist([1,2],[{(1,2):1},{(2,3):2, (1,3):3}])
dist2.scalars

[1, 2]

In [11]:
dist2.dicts

[{(1, 2): 1}, {(2, 3): 2, (1, 3): 3}]

and in fact, for a dictionary ```dict```, ```alpha(dict)``` is the exact same thing as ```dist([1],[dict])```.

Every time a ```dist``` object is created, the scalars and dictionaries are condensed as much as possible, so that only unique dictionaries are listed, and only non-zero scalars are listed:

In [12]:
dict1 = {(1,2):1}
dict2 = {(2,3):2}

dist3 = dist([1,2,0],[dict1,dict1,dict2])

dist3.scalars

[3]

In [13]:
dist3.dicts

[{(1, 2): 1}]

The equality operator ```==``` of course pays no heed to the order in which the scalars and their associated dictionaries are listed:

In [14]:
dist([1,2,0],[dict1,dict2,dict3]) == dist([2,1],[dict2,dict1])

True

As shown above, scalars (that is, any Python object ```x``` for which ``` isinstance(x,numbers.Number) ```
returns ```True```) and ```dist``` objects can be mixed freely in arithmetic expressions.  When two such occur together, the scalar ```c``` is treated as the distribution ```c*alpha({ })``` that is, as ```dist([c],[{ }])``` where ```{ }``` is the empty dictionary (since ```dist([1],[{ }])```, i.e. $\alpha(0)$, is a multiplicative identity for $\text{Dist}(U_n)$).  And, the scalar ```0``` is equivalent to ```dist([ ],[ ])```.

Two derived attributes of a ```dist``` object may be useful to access.  The ```dimension```:

In [15]:
dist2.dimension

3

This is simply the smallest $n$ for which ```dist2``` occurs as an element of $\text{Dist}(U_n)$.  Also 

In [16]:
abs(dist2)

5

which is the smallest $m$ for which ```dist2``` occurs as an element of $\text{Dist}_m(U_n)$ (see proposition 3.1).

For computations in characteristic $p>0$, there is a class function ```mod_p``` for taking a distribution (i.e. its scalars) mod $p$:

In [17]:
dist4 = dist([7,11,15],[dict1,dict2,dict3])
new_dist = dist4.mod_p(5)                        #take the distribution dist4 mod p = 5
new_dist.scalars

[2, 1]

In [18]:
new_dist.dicts

[{(1, 2): 1}, {(2, 3): 2}]

It is incumbent upon the user to make sure that taking mod $p$ actually makes sense for the scalars of a ```dist``` object (they will always be integers, unless otherwise introduced by the user).  It is recommend that you call ```mod_p``` early and often, since doing so will make scalars shrink and sometimes vanish, drastically speeding up computations in many instances.

Finally, be aware that $\text{Dist}(U_n)$ is by nature a computationally expensive algebra.  Distributions can get very large and expensive very quickly (the following computation takes around 10 seconds, and ends up with a scalar list of length 546):

In [19]:
dist1 = alpha({(1,2):1}) + alpha({(3,4):1}) + 3*alpha({(4,5):2})
new_dist = dist1**15
len(new_dist.scalars)

546

So don't be surprised if performance starts to suffer earlier than expected.   Again, when working mod $p$, taking ```mod_p``` as often as possible can make these computations much more feasible.
